In [ ]:
import numpy as np
import pyarrow as pa
import pyarrow
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import pyarrow
targets = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
test = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/test.csv')
stock_ids = list(sorted(targets.stock_id.unique()))
ntimeids = targets.time_id.nunique()
timeids = sorted(targets.time_id.unique())
timeid_to_ix = {time_id:i for i,time_id  in enumerate(timeids)}
stock_id_to_ix = {stock_id:i for i,stock_id  in enumerate(stock_ids)}
ntimeids = targets.time_id.nunique()



In [ ]:
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
def book_features(df):
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return).fillna(0)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return).fillna(0)
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    return df
    

In [ ]:
import numpy as np

ncolumns = 18
nstocks = len(stock_ids)
nseq =  600
arr =np.memmap('./data.array',dtype='float32',mode='w+',shape=(ntimeids,nstocks,nseq,ncolumns))

In [ ]:

for stock in stock_ids:
    stock_ix = stock_id_to_ix[stock]
    
    #loads trades for the stock
    tds = pyarrow.parquet.ParquetDataset('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/',filters=[('stock_id','=',f'{stock}')],validate_schema=True)
    Trade = tds.read().to_pandas()
    Trade['log_return_trade'] = Trade.groupby('time_id')['price'].apply(log_return).fillna(0)
    
    #loads book
    bds = pyarrow.parquet.ParquetDataset('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/',filters=[('stock_id','=',f'{stock}')],validate_schema=True,)
    Book  =bds.read()
    Book = Book.to_pandas()
    Book = book_features(Book)
    
    #Join trade and book, I call it Asof like an asof join
    Asof = pd.merge(Book,Trade,on=['seconds_in_bucket','time_id'],how='left')
    Asof = Asof.sort_values(['time_id','seconds_in_bucket']).fillna(0)
    Asof = Asof.rename(columns={'stock_id_x':'stock_id'})
    del Asof['stock_id']
    del Asof['stock_id_y']
    
    for time_id in timeids:
        target =targets[(targets.stock_id==stock) & (targets.time_id==time_id)]['target']
        if len(target)>0:
            time_ix = timeid_to_ix[time_id]
            asof_input = Asof[Asof.time_id==time_id]
            del asof_input['time_id']
            arr[time_ix,stock_ix,:len(asof_input),:] = asof_input.values
    

            

In [ ]:
arr.flush()